# Microwave Point to Point Link Recieved Signal Strength Calculations

#### Project Name:
#### Project Number:
##### Calculated By:
###### Date:
##### Checked By:
###### Date:

#### Import the Python libraries needed to perform the calculations:

In [1]:
from math import *

## Determine Losses Between Antenna and Transmitter or Reciever

$L_{cable} = L_0 * (d_{cable} / 100)$

$L_{tx}$ and $L_{rx} = L_{cable} + L_{misc.}$

$L_0$ = cable signal loss in dB per 100 ft from manufacturer data sheet <br>
$d_{cable}$ = length of cable in feet from antenna to transmitter or reciever <br>
$L_{misc.}$ = all combiner/splitter or other losses in dB between antenna and transmitter or reciever



In [9]:
def cable_loss(L_0, d, L_misc):
    L_cable = L_0 * (d / 100.0)
    return L_cable + L_misc

In [ ]:
#problem calculations here

## Calculate Free Space Loss (FSL)

$FSL = 96.6 + 20 * log (D) + 20 * log (f)$

$FSL$ = free space loss in decibels <br>
$D$ = distance between antenna in miles <br>
$f$ = frequency in GHz

In [3]:
def free_space_loss(D, f):
    return 96.6 + 20 * math.log10(D) + 20 * log10(f)

In [4]:
#problem calculations here

## Set Mutipath Fade Margin (MFM)
### Using the Barnett & Vigants Method:

$MFM = -10 * log[(i - PR) / (2.5*10^{-6} * a * b * f * D^3)]$

$PR$ = path avaliability percentage <br>
$a$ = terrain factor <br>
$b$ = climate factor <br>
$f$ = frequency in GHz <br>
$D$ = path length in miles between antenna<br>

**Notes:**
- **a** values vary according to terrain type: {**4**: for very smooth terrain (water, flat desert), **1**: average terrain with some roughness, **0.25**: for mountains and very rough terrain}
- **b** values vary according to climate: {**1**: for humid climates, **0.5**: for average climate, **0.25**: for dry climate}
- Select a **PR** based on the intended link availability percentage.

In [5]:
def multipath_fade_margin(PR, a, b, f, D):
    return (-10 * math.log10((1 - (PR / 100)) / (2.5 * (10.0 ** -6) * a * b * f * (D ** 3))))

In [6]:
#problem calculations here

## Calculate Recieved Signal Strength (RSL)

$RSL = P_O - L_{tx} + G_{atx} - FSL - MFM - RA - L_{rc} + G_{rc} - L_m - L_{atm} - A_{ve}$

$P_O$ = transmitter power output in dBm <br>
$L_{tx}$ = losses between transmitter and transmit antenna in dB <br>
$G_{atx}$ = transmit antenna gain in dB <br>
$A_{ve}$ = vegitation attenuation in dB (use $A_{et}$ for single vegitative obstruction)<br>
$FSL$ = free space loss in dB <br>
$MFM$ = multipath fade margin in dB <br>
$RA = \gamma_R * D$ = rain attenuation in dB <br>
$L_{rc}$ = losses between reciever and recieve antenna in dB <br>
$G_{rc}$ = recieve antenna gain in dB <br>
$L_m$ = micellaneous losses (obstacles, antenna misalignment, antenna aging, etc.) in dB <br>
$L_{atm}$ = atmospheric attenuation in dB

#### Rain Attenuation (from "Recommendation ITU-R P.838-2"):
$\gamma_R = k * R^\alpha$

$\gamma_R$ = specific rain attenuation in dB/km <br>
$k$ & $\alpha$ = frequency specific coefficients from ITU-R P.838-2 tables

#### Atmospheric Attenuation (from "Recommendation ITU-R P.676-5"):
Simplified formula from Annex 2 for Terrestrial Paths (1 - 350 GHz), using Figure 5 values for specific attenuation:

$L_{atm} = \gamma * r_0 = (\gamma_o + \gamma_w) * r_0$

$\gamma$ = total attenuation by atmosphere in dB/km <br>
$\gamma_o$ = attenuation due to dry air in dB/km <br>
$\gamma_w$ = attenuation due to water in atmosphere in dB/km <br>
$r_0$ = link distance in km

#### Vegitation Attenuation (from "Recommendation ITU-R P833-4"):
##### For non-single vegetative obstruction: 

$A_{ve} = A_m * [1 - exp ((-d * \gamma)/A_m)]$

$d$ = length of path within woodland in meters <br>
$\gamma$ = specific attenuation for very for vegatative paths in dB/meter <br>
$A_m$ = maximum attenuation for one terminal within a specific type and depth of vegetation in dB <br>

$A_m = A_1 * f^\alpha$

$f$ = frequency in MHz <br>
$A_1$ & $\alpha$ depend on the vegitation type, density, and other factors. ITU-R P833-4 provide experimental values.

##### For single vegetative obstruction at or below 3 GHz:
$A_{et} = d * \gamma$


**Notes**: 
- $L_m$ is assumed to be **6.0** dB to account for attenna misalignment and system aging + **Obstacle losses**
- Obstacle losses (knife-edge approximation): {**0 dB**: no objects in 60% Fresnel Zone, **6 dB**: tip of object in middle of LOS, **16 dB** for tip of object at top of first Fresnel Zone, **20 dB**: for object tip above first Fresnel Zone}




In [1]:
"""Equations required to perform RSL calculations"""

def rain_attenuation(k, alpha, D, R, D_in_miles = True):
    if D_in_miles:
        D = D * 1.60934 #convert distance input in miles to km
    delta_R = k * R ** alpha #specific rain attenutation dB/km
    return float(delta_R * D)

def atmospheric_attenuation(atten_oxy, atten_water, D, D_in_miles = True):
    if D_in_miles:
        D = D * 1.60932 #convert distance input in miles to km
    return float((atten_oxy + atten_water) * D)

def veg_atten_multi(path_length_veg, specific_atten, A_1, alpha, frequency, D_in_miles = True, f_in_GHz = True):
    if D_in_miles:
        D = D * 1609.34 #convert distance input in miles to meters
    if f_in_GHz:
        frequency = frequency / 1000.0 #convert frequency in GHz to MHz
    max_atten = A_1 * frequency ** alpha
    return float(max_atten * (1 - math.exp((-d * specific_atten) / max_atten)))

def veg_atten_single(path_length_veg, gamma, D_in_miles = True):
    if D_in_miles:
        D = D * 1609.34 #convert distance input in miles to meters
    return float(path_length_veg * gamma)

def RSL(P_O, L_tx, G_atx, FSL, MFM, RA, L_rc, G_rc, L_m, L_atm, A_ev):
    return float(P_O - L_tx + G_atx - FSL - MFM - RA - L_rc + G_rc - L_m - L_atm - A_ve)

def check_system(recieved_signal_strength, min_recieve_sensitivity):
    check = recieved_signal_strength > min_recieve_sensitivity
    print "Recieved Signal Strength is adequate:", check 

In [8]:
#problem caluculations here